# A Hypermetric Topic Model of Top 40 Pop/Rock Harmony
##aka completely incomplete awesome nonsense

In [213]:
import pandas as pd
import numpy as np
import seaborn as sns
import logging, os, json, glob
import gensim

from gensim import corpora, models, similarities
from collections import defaultdict
from pprint import pprint

# OPTIONS
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.level = logging.WARN
basedir = """C:\chords\McGill-Billboard\out"""

THRESH = False
NGRAMS = True
BARGRAMS = True # much better than ngrams
TFIDF = False


In [214]:
# LOAD FILES AND CREATE CHORD TOKENS
# (currently removing time signature information)
corpus = []
sigs = []
processed_corpus = []
for fn in glob.glob(os.path.join(basedir, "*.txt")):
    with open(fn, 'r') as f:
        try: # try to convert to timesig; discard incorrect formats
            sig = int(f.readline()[-4:][-4])
            sigs.append(sig)
        except ValueError:
            continue
        corpus.append([line.strip().lower()
                       for line in f
                       if not line.startswith("Time")])

processed_corpus = [[] for x in xrange(len(corpus))]
print "Loaded {} songs...".format(len(corpus))

 

Loaded 876 songs...


In [215]:
# Make bar- and hyperbar-length tokens
if BARGRAMS:
    # 2 bars, and then (usually) 3 or 4 depending on hypermeter
    for multiplier in [lambda: 2, lambda: sigs[i], lambda: sigs[i] * 2]:
        for i, song in enumerate(corpus):
            bargrams = []
            currentseq = []
            
            split = [chord.split(",") for chord in song]
            durs = [(int(chord[0]), int(chord[1])) for chord in split]
            
            # multiples of top number of timesig
            bar_beats = sigs[i] * multiplier()
            total_beats = 0

            for j, chord in enumerate(song):
                chord_beats = durs[j][1]
                if total_beats < bar_beats:
                    currentseq.append(chord)
                    total_beats += chord_beats
                if total_beats >= bar_beats:
                    # GET RID OF SINGLETONS
                    bargrams.append(currentseq[:])
                    currentseq = []
                    total_beats = 0
            newgrams = ["; ".join(seq) for seq in bargrams]
            processed_corpus[i].extend(newgrams)







##Preprocessing
### N-Grams
- Trying out 2, 3, and 4-grams simultaneously
- using the idea that having all of these simultaneously represents a "cognitive context"
- but this pollutes the topics...what's the solution?
###Bargrams
- N-grams are boring. Let's do grams of variable length where the beats sum to a one, two, etc. measures!
###Thresholding
- We can also try thresholding, but that doesn't really work
###TF-IDF
- Never do this: the frequent stuff, while useless in language, means that we get musical output that doesn't have the useful common chord progressions we all know and love. Music doesn't really have an "ordinary language". Things that are "roughage" in natural language are content in music!



In [216]:
# PREPROCESS CORPUS

# add ngrams to each song
if NGRAMS:
    for i, song in enumerate(corpus):
        song_grams = []
        for j in xrange(2, 5):
            grams = zip(*[song[k:] for k in xrange(j)])
            song_grams.extend(["; ".join(gram) for gram in grams])        
        processed_corpus[i].extend(song_grams)

if THRESH:
    freqs = defaultdict(int)
    for song in processed_corpus:
        for chord in song:
            freqs[chord] += 1
    processed_corpus = [[chord 
               for chord in song 
               if freqs[chord] > 1] 
               for song in processed_corpus]


In [217]:
dictionary = corpora.Dictionary(processed_corpus)
print dictionary
dictionary.save("""C:/mcgillout/billboardgensim.dict""")
corpus_bow = [dictionary.doc2bow(song) for song in processed_corpus]
corpora.MmCorpus.serialize("""C:/mcgillout/billboardgensim.mm""", corpus_bow)

if TFIDF:
    tfidf = models.TfidfModel(corpus_bow)
    corpus_bow = tfidf[corpus_bow]

Dictionary(58810 unique tokens: [u'0,4,e,maj; 0,4,g,sus4(b7,9,13); 0,4,e,maj; 0,4,a,maj; 0,4,a,maj; 0,4,c,min; 0,4,c,min; 0,4,c,min7', u'3,1,f,maj; 0,1,a-,maj', u'0,4,d,maj; 0,4,a,min; 0,4,d,maj', u'0,4,g,7; 0,1,c,maj7; 1,3,c,maj; 0,1,c,maj7; 1,3,c,maj; 0,1,d,min6; 1,3,d,min7', u'0,2,f,maj; 2,2,c,maj; 0,2,f,maj; 2,2,c,maj; 0,2,f,maj; 2,2,c,maj; 0,4,g,maj; 0,4,c,maj; 0,2,f,maj; 2,2,c,maj; 0,2,f,maj; 2,2,c,maj; 0,4,g,maj']...)


#Hyperparameters
###Alpha
Sparsity of per-doc topic weights: higher <==== similar content in docs
###Eta
Sparsity of topic distributions:   higher ====> similar content in topics

In [220]:
%time lda = gensim.models.wrappers.LdaVowpalWabbit("C:/users/dakota/vw/vw.exe", corpus=corpus_bow, id2word=dictionary, num_topics=25, passes = 4000, alpha = 0.99, eta = 0.999)#, iterations=1000, passes=5)

Wall time: 12min 8s


In [221]:
# 25 topics, with ngrams
for i, t in enumerate([topic.replace("+",u'\n') 
                       for topic 
                       in lda.print_topics(-1)]):
    print "TOPIC {}: {}".format(i + 1, t)
    print "\n----------------------------------------------------------------\n"


TOPIC 1: 0.021*0,4,c,min7; 0,4,c,min7 
 0.011*0,4,c,min7; 0,4,c,min7; 0,4,c,min7; 0,4,c,min7 
 0.010*0,4,c,min7; 0,4,c,min7; 0,4,c,min7 
 0.003*0,4,c,min7 
 0.003*0,4,f,min7; 0,4,f,min7 
 0.002*0,4,c,min7; 0,4,c,min7; 0,4,f,min7; 0,4,f,min7 
 0.001*0,4,c,min7; 0,4,f,min7 
 0.001*0,4,c,min7; 0,2,c,min9; 2,2,c,min7 
 0.001*0,4,f,min7; 0,4,c,min7 
 0.001*0,4,c,min7; 0,4,c,min7; 0,4,f,min7

----------------------------------------------------------------

TOPIC 2: 0.011*0,2,c,maj; 2,2,f,maj 
 0.007*2,2,f,maj; 0,2,c,maj 
 0.006*0,2,c,maj; 2,2,f,maj; 0,2,c,maj; 2,2,f,maj 
 0.005*2,2,f,maj; 0,2,c,maj; 2,2,f,maj 
 0.005*0,2,c,maj; 2,2,f,maj; 0,2,c,maj 
 0.004*2,2,f,maj; 0,2,c,maj; 2,2,f,maj; 0,2,c,maj 
 0.003*0,2,c,maj; 2,2,f,maj; 0,4,c,maj 
 0.002*2,2,f,maj; 0,4,c,maj 
 0.002*0,2,c,maj; 2,2,f,maj; 0,4,g,maj 
 0.002*0,4,g,maj; 0,2,c,maj; 2,2,f,maj

----------------------------------------------------------------

TOPIC 3: 0.002*0,4,c,min7; 0,4,f,13 
 0.001*0,4,c,min7; 0,4,f,13; 0,4,c,min7; 0,4

In [219]:
# ngrams
for i, t in enumerate([topic.replace("+",u'\n') 
                       for topic 
                       in lda.print_topics(-1)]):
    print "TOPIC {}: {}".format(i + 1, t)
    print "\n----------------------------------------------------------------\n"


TOPIC 1: 0.005*0,4,c,13; 0,4,c,13 
 0.004*0,4,c,13; 0,4,c,13; 0,4,c,13; 0,4,c,13 
 0.003*0,4,c,13; 0,4,c,13; 0,4,c,13 
 0.001*0,1,c,min7/b7; 1,1,c,min7/13 
 0.001*1,1,c,min7/13; 2,2,g,aug(b7) 
 0.001*0,1,c,min7/b7; 1,1,c,min7/13; 2,2,g,aug(b7) 
 0.001*0,4,c,min7; 0,4,f,7 
 0.001*0,3,c,5(b7); 3,1,d-,5 
 0.001*2,2,g,aug(b7); 0,2,g,aug(b7) 
 0.001*2,2,g,aug(b7); 0,2,g,aug(b7); 2,2,c,min9

----------------------------------------------------------------

TOPIC 2: 0.025*0,4,c,7; 0,4,c,7 
 0.011*0,4,c,7; 0,4,c,7; 0,4,c,7 
 0.011*0,4,c,7; 0,4,c,7; 0,4,c,7; 0,4,c,7 
 0.006*0,4,f,7; 0,4,f,7 
 0.005*0,4,f,7; 0,4,c,7 
 0.005*0,4,c,7; 0,4,f,7 
 0.003*0,4,f,7; 0,4,c,7; 0,4,c,7 
 0.002*0,4,c,7; 0,4,g,7 
 0.002*0,4,f,7; 0,4,f,7; 0,4,c,7; 0,4,c,7 
 0.002*0,4,g,7; 0,4,f,7

----------------------------------------------------------------

TOPIC 3: 0.002*0,4,c,min/13; 0,2,a-,maj7; 2,2,g,7 
 0.002*0,4,c,min; 0,4,c,min/b7 
 0.002*0,2,f,maj; 2,2,c,maj 
 0.002*0,4,c,maj; 0,4,g,maj 
 0.002*0,2,f,maj/5; 2,2,c,

In [212]:
# no ngrams
for i, t in enumerate([topic.replace("+",u'\n') 
                       for topic 
                       in lda.print_topics(-1)]):
    print "TOPIC {}: {}".format(i + 1, t)
    print "\n----------------------------------------------------------------\n"


TOPIC 1: 0.005*0,4,c,min/13; 0,2,a-,maj7; 2,2,g,7 
 0.005*0,4,c,min; 0,4,c,min/b7 
 0.005*0,4,c,min; 0,4,c,min/b7; 0,4,c,min/13; 0,2,a-,maj7; 2,2,g,7 
 0.003*0,2,c,maj; 2,2,f,maj; 0,2,b-,maj; 2,2,f,maj 
 0.003*0,2,c,maj; 2,2,f,maj; 0,4,c,maj 
 0.003*0,2,c,min; 2,2,f,maj6; 0,2,c,min; 2,2,f,maj6 
 0.002*0,4,c,min; 0,4,e-,maj 
 0.002*0,4,c,min; 0,4,c,min/b7; 0,4,c,min/13; 0,2,a-,maj7; 2,2,g,7; 0,4,c,min; 0,4,c,min/b7; 0,4,c,min/13; 0,2,a-,maj7; 2,2,g,7 
 0.002*0,4,f,min; 0,2,a-,maj; 2,2,g,7 
 0.002*0,4,c,min; 0,4,e-,maj; 0,4,f,min; 0,2,a-,maj; 2,2,g,7

----------------------------------------------------------------

TOPIC 2: 0.024*0,4,c,min7; 0,4,c,min7 
 0.009*0,4,c,min7 
 0.007*0,4,,; 0,4,, 
 0.006*0,4,c,min7; 0,4,c,min7; 0,4,c,min7; 0,4,c,min7 
 0.005*0,4,f,min7; 0,4,f,min7 
 0.003*0,4,g,min; 0,2,e-,maj; 2,2,f,maj 
 0.003*0,4,c,1; 0,4,c,1 
 0.003*0,4,,; 0,4,,; 0,4,,; 0,4,, 
 0.003*0,4,c,min7; 0,4,c,min7; 0,4,f,min7; 0,4,f,min7 
 0.002*0,4,c,min7; 0,4,c,min7; 0,4,c,min7; 0,4,c,min7; 0,

#What next?
I want to try the S&S suffix tree thing for chords!
- correlated topic model
- that one variant made for recurrent hierarchical temporal patterns
- dynamic topic model, wallach's ngrams
    - http://dirichlet.net/pdf/wallach06topic.pdf
    - http://jgoodwin.net/blog/experimenting-with-dynamic-topic-models/
    - bleicode: https://code.google.com/p/princeton-statistical-learning/downloads/list
        - formatter for blei's code: https://github.com/JoKnopp/text2ldac